In [609]:
# Импорты необходимых библиотек
import pandas as pd  # Для работы с данными (DataFrame, чтение CSV)
import seaborn as sns  # Для визуализации (lineplot в plot_history)
import regex  # Для регулярных выражений (валидация имени, лучшая поддержка Unicode)
from dateutil import parser  # Для парсинга дат в fix_date
from matplotlib import pyplot as plt  # Для графиков в plot_history
from datetime import datetime  # Для работы с датами и временем (now() в _add_operation)
from IPython.display import display  # Для отображения DataFrame в Jupyter (опционально)

# Отключаем предупреждения Pandas для чистоты вывода
pd.options.mode.chained_assignment = None


In [ ]:
# ---
# Задание 1: Реализация базового класса Account
# ---

class Account:
    """
    Базовый класс, моделирующий банковский счёт.
    Выполняет базовые операции, ведёт учёт истории, предоставляет аналитику.
    """
    
    # Приватный атрибут класса для генерации уникальных номеров счёта (отсчёт начинается с 1000, этап 1)
    _account_counter = 1000

    def __init__(self, account_holder: str, balance: float = 0, account_number: str= None):
        """
        Конструктор класса (этап 2). 
        Автоматически формирует номер счёта 'ACC-XXXX'.
        Валидирует имя владельца и баланс.
        """
        # Валидация имени владельца (п.4 задания 2)
        self._account_holder = self._validate_holder_name(account_holder)
        # Валидация начального баланса (не может быть отрицательным, п.4 задания 2)
        self._balance = self._validate_balance(balance, 'создание счёта')
        
        # Автоматическая генерация номера счёта (этап 1)
        if account_number:
            
            self._account_number = account_number
        else:
            self._account_number = f'ACC-{Account._account_counter}'
            Account._account_counter += 1
        
        # Инициализация истории операций (список структурированных словарей, этап 1)
        self._operations_history = []
        # Атрибут для типа счёта (для наследования, этап 4 задания 2)
        self.account_type = None
        # Вывод для демонстрации создания счёта
        print(f"Счёт {self._account_number} для {self._account_holder} успешно создан. Баланс: {self._balance} у.е.")

    @staticmethod
    def _validate_holder_name(account_holder: str) -> str:
        """
        Валидация имени владельца (п.4 задания 2).
        Формат: "Имя Фамилия" с заглавных букв, кириллица или латиница.
        Вызывает исключение при несоответствии.
        """
        account_holder = account_holder.title().strip()  # Нормализация: заглавные буквы, удаление лишних пробелов
        pattern = r'^[A-ZА-ЯЁ][a-zа-яё]*\s[A-ZА-ЯЁ][a-zа-яё]*$'  # Паттерн для "Имя Фамилия" (заглавные первые буквы, кириллица/латиница)
        if not isinstance(account_holder, str) or not regex.match(pattern, account_holder):
            raise ValueError("Неверный формат имени владельца. Ожидается 'Имя Фамилия' с заглавных букв (кириллица или латиница).")
        return account_holder

    @staticmethod
    def _validate_balance(value: float, operation_type: str) -> float:
        """
        Валидация начального баланса или суммы (п.4 задания 2).
        Должна быть положительным числом, иначе исключение.
        """
        if not isinstance(value, (int, float)) or value < 0:
            raise ValueError(f'Операция "{operation_type}" не выполнена: баланс/сумма не может быть отрицательной.')
        return float(value)
    
    @staticmethod
    def _validate_amount(amount: float):
        """
        Валидация суммы операции (п.4 задания 2).
        Должна быть положительной, иначе исключение.
        """
        if not isinstance(amount, (int, float)) or amount <= 0:
            raise ValueError("Сумма операции должна быть положительным числом.")

    @staticmethod
    def fix_date(df, date_col='date', raw_col='raw_date'):
        """
        Исправление аномалий в датах (перепутанные день/месяц) на основе разницы с предыдущей датой (часть очистки, задание 3).
        """
        df_fixed = df.copy() 
    
        df_fixed['time_diff_days'] = df_fixed[date_col].diff().dt.days
    
        ANOMALY_THRESHOLD = -10 
        anomalous_indices = df_fixed[df_fixed['time_diff_days'] < ANOMALY_THRESHOLD].index
    
        if anomalous_indices.empty:
            return df_fixed.drop(columns=['time_diff_days'], errors='ignore')

        for index in anomalous_indices:
            raw_date_str = df_fixed.loc[index, raw_col]
        
            try:
                new_date = parser.parse(raw_date_str, fuzzy=True, dayfirst=True)
            
                df_fixed.loc[index, 'temp_date'] = new_date
                temp_diff = df_fixed['temp_date'].fillna(df_fixed[date_col]).diff().dt.days
            
                if temp_diff.loc[index] >= 0:
                    df_fixed.loc[index, date_col] = new_date
                else:
                    df_fixed.loc[index, date_col] = pd.NaT

            except Exception:
                df_fixed.loc[index, date_col] = pd.NaT
            
        df_fixed = df_fixed.drop(columns=['time_diff_days', 'temp_date'], errors='ignore')
        df_fixed = df_fixed.dropna(subset=[date_col]) 
    
        return df_fixed.sort_values(by=date_col) 
    
    def load_history_from_file(self, filepath: str):
        """
        Загрузка истории из файла (CSV или JSON, задание 3).
        Фильтрует по аккаунту, очищает данные, добавляет новые операции, обновляет баланс.
        """
        if self.account_type is None:
            print("account_type не определён. Метод доступен только для подклассов (CheckingAccount/SavingsAccount).")
            return
        
        print(f"\nЗагрузка истории для {self._account_number} из {filepath}...")
        
        try:
            if filepath.endswith('.csv'):
                df = pd.read_csv(filepath)
            elif filepath.endswith('.json'):
                df = pd.read_json(filepath)
            else:        
                raise ValueError('Неверный формат файла (ожидается .csv или .json).')
            
        except Exception as e:
            print(f"Ошибка при чтении файла: {e}")
            return
       
        # Фильтрация данных по номеру аккаунта и типу счета (файл общий для всех, задание 3)
        df_filtered = df[
            (df['account_number'] == self._account_number) &
            (df['account_type'] == self.account_type)
        ]

        if df_filtered.empty:
            print(f"Транзакции для {self._account_number} ({self.account_type}) не найдены в файле.")
            return
        
        # Преобразование DataFrame в список словарей для clean_history
        raw_transactions = df_filtered.to_dict('records')
        cleaned_transactions = self.clean_history(raw_transactions)

        if not cleaned_transactions:
            print("После очистки не осталось валидных транзакций для загрузки.")
            return
        
        # Проверить, нет ли этих операций уже в истории (по дате и сумме)
        new_ops_added = 0
        existing_signatures = {(op['datetime'], op['amount']) for op in self._operations_history}
        
        for op in cleaned_transactions:
            signature = (op['datetime'], op['amount'])
            if signature not in existing_signatures:
                self._operations_history.append(op)
                new_ops_added += 1

        print(f"Добавлено {new_ops_added} новых транзакций в историю.")
        
        # Обновление баланса до последнего значения в файле (задание 3)
        if new_ops_added > 0:
            # Сортируем всю историю (старую + новую)
            self._operations_history.sort(key=lambda op: op['datetime'])
            # Берем баланс из самой последней транзакции
            self._balance = self._operations_history[-1]['balance_after']
            print(f"Баланс счёта {self._account_number} обновлён до: {self._balance} у.е. (на основе загруженных данных)")
    
    def _get_valid_operation_types(self):
        """
        Вспомогательный метод для clean_history (задание 3).
        Определяет валидные типы операций для этого типа счёта (deposit, withdraw для CheckingAccount).
        """
        return ['deposit', 'withdraw']
    
    def clean_history(self, raw_transactions: list[dict]) -> list[dict]:
        """
        Очистка списка транзакций от "грязных" данных перед записью (задание 3).
        Проверяет account_number, account_type, operation (динамично), amount >0, status, даты (парсинг и fix).
        Возвращает список очищенных словарей.
        """
        print(f"Запущена очистка {len(raw_transactions)} транзакций...")
        df = pd.DataFrame(raw_transactions)
        
        if df.empty:
            print("Данные пусты после фильтрации.")
            return []
        
        # Проверка account_number на паттерн
        account_pattern = r'^ACC-\d{4,}$'
        df = df[df['account_number'].str.match(account_pattern, na=False)]
        
        # Проверка account_type (только валидные)
        valid_account_types = ['checking', 'savings']
        df = df[df['account_type'].isin(valid_account_types)]
        
        # Проверка operation (динамично по типу счёта)
        valid_operations = self._get_valid_operation_types()
        df = df[df['operation'].isin(valid_operations)]
        
        # Проверка amount (>0, numeric)
        df = df.dropna(subset=['amount'])
        df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
        df = df.dropna(subset=['amount'])
        df = df[df['amount'] > 0]
        
        # Проверка status (только success, fail — invalid)
        valid_statuses = ['success']
        df = df[df['status'].isin(valid_statuses)]
        
        # Парсинг и исправление дат
        df['raw_date'] = df['date']
        def parse_date(date_str):
            try:
                return parser.parse(date_str, fuzzy=True)
            except:
                return pd.NaT

        df['date'] = df['date'].apply(parse_date)
        df = df.dropna(subset=['date'])
        df_corrected_groups = df.groupby(['account_number', 'account_type']).apply(self.fix_date)
        df_cleaned = df_corrected_groups.reset_index(drop=True)
        
        # Адаптация ключей для истории операций
        cleaned_data = df_cleaned.to_dict('records')
        for op in cleaned_data:
            op['datetime'] = op['date']
            op['type'] = op['operation']
            del op['date']
            del op['operation']
        
        print(f"Очистка завершена. Валидных транзакций: {len(cleaned_data)}")
        return cleaned_data

    def _add_operation(self, op_type: str, amount: float, status: str, balance_after: float = None):
        """
        Добавление операции в историю (этап 1).
        Каждая операция — словарь с типом, суммой, датой/временем, балансом после, статусом.
        """
        if balance_after is None:
            balance_after = self._balance
            
        operation = {
            "datetime": datetime.now(),  # Дата и время (datetime.now(), этап 2)
            "type": op_type,  # Тип операции ('deposit' или 'withdraw')
            "amount": amount,  # Сумма
            "balance_after": round(balance_after, 2),  # Баланс после (округлён до 2 знаков)
            "status": status  # 'success' или 'fail'
        }
        self._operations_history.append(operation)

    def deposit(self, amount: float):
        """
        Пополнение счёта (этап 2).
        Проверяет сумму >0, обновляет баланс, добавляет в историю.
        При отрицательной сумме — исключение и 'fail' в истории.
        """
        try:
            self._validate_amount(amount)
            self._balance += amount
            self._add_operation('deposit', amount, 'success')
            print(f'Счёт {self._account_number}: депозит {amount} у.е. выполнен. Новый баланс: {self._balance} у.е.')
        except ValueError as e:
            self._add_operation('deposit', amount, 'fail')
            print(f'Счёт {self._account_number}: ошибка депозита. {e}')

    def withdraw(self, amount: float):
        """
        Снятие средств (этап 2).
        Проверяет сумму >0 и достаточность средств, обновляет баланс, добавляет в историю.
        При недостатке — 'fail' в истории, без изменения баланса.
        """
        try:
            self._validate_amount(amount)
            if self._balance < amount:
                self._add_operation('withdraw', amount, 'fail')
                print(f'Счёт {self._account_number}: ошибка снятия. Недостаточно средств. Запрошено: {amount}, Доступно: {self._balance}')
                return
            self._balance -= amount
            self._add_operation('withdraw', amount, 'success')
            print(f'Счёт {self._account_number}: снятие {amount} у.е. выполнено. Новый баланс: {self._balance} у.е.')
        except ValueError as e:
            self._add_operation('withdraw', amount, 'fail')
            print(f'Счёт {self._account_number}: ошибка снятия. {e}')

    def get_balance(self) -> float:
        """
        Возврат текущего баланса (этап 2).
        """
        return self._balance

    def get_history(self) -> list[dict]:
        """
        Возврат истории операций (этап 2).
        Возвращает список структурированных словарей.
        """
        return self._operations_history

    def plot_history(self):
        """
        Визуализация истории операций (этап 3, для 8+ баллов).
        Создаёт DataFrame из истории, группирует по месяцам, строит линейный график с подписями.
        """
        if not self._operations_history:
            raise ValueError('История операций пуста.')
        df = pd.DataFrame(self._operations_history)
        df = df[df['status'] == 'success']
        df['datetime'] = pd.to_datetime(df['datetime'])
        
        df['year_month'] = df['datetime'].dt.strftime('%m').astype(int)
        add_months = df.groupby('year_month', as_index=False)['balance_after'].last()

        fig, ax = plt.subplots(figsize=(8, 6))
        sns.lineplot(data=add_months, x='year_month', y='balance_after', marker='o', ax=ax)

        for i, row in add_months.iterrows():
            ax.text(row['year_month'], row['balance_after'], f'{row['balance_after']:.0f}',
                    ha='center', va='baseline', fontsize=10)
        ax.set_title('Динамика счёта по месяцам')
        ax.set_xlabel('Год-Месяц')
        ax.set_ylabel('Баланс после операций')
        plt.show()

    def get_large_operations(self, n: int = 5) -> list[dict]:
        """
        Анализ истории: последние n крупных операций по сумме и дате (п.5 задания 2).
        Сортирует по дате (desc) и сумме (desc), возвращает список.
        """
        success_ops = [op for op in self._operations_history if op['status'] == 'success']
        
        if not success_ops:
            return []
        
        # Сортировка: по дате desc, сумме desc
        sorted_ops = sorted(success_ops, key=lambda op: (op['datetime'], op['amount']), reverse=True)
        
        return sorted_ops[:n]



In [ ]:

# ---
# Задание 2: Реализация наследования (этап 4)
# ---

class CheckingAccount(Account):
    """
    Расчётный счёт (CheckingAccount), наследуется от Account (этап 4).
    Хранит атрибут account_type = 'checking'.
    """
    def __init__(self, account_holder: str, balance: float = 0, account_number: str = None):
        super().__init__(account_holder, balance)
        self.account_type = 'checking'
        print(f'-> Создан расчётный счёт (CheckingAccount): {self._account_number} для {self._account_holder}')


In [ ]:

class SavingsAccount(Account):
    """
    Сберегательный счёт (SavingsAccount), наследуется от Account (этап 4).
    Хранит атрибут account_type = 'savings'.
    Реализует apply_interest (п.2) и ограничение на withdraw (п.3).
    """
    def __init__(self, account_holder: str, balance: float = 0, account_number: str = None):
        super().__init__(account_holder, balance)
        self.account_type = 'savings'
        print(f'-> Создан сберегательный счёт (SavingsAccount): {self._account_number} для {self._account_holder}')
        
    def withdraw(self, amount: float):
        """
        Переопределённый метод снятия (п.3 задания 2).
        Нельзя снять больше 50% от баланса, иначе исключение и 'fail' в истории.
        """
        try:
            self._validate_amount(amount)
            limit = self._balance * 0.5
            if amount > limit:
                self._add_operation('withdraw', amount, 'fail')
                print(f'Снятие запрещено. Нельзя снять больше 50% от баланса (лимит {limit:.2f} у.е.). Запрошено: {amount}.')
                return
            super().withdraw(amount)
        except ValueError as e:
            self._add_operation('withdraw', amount, 'fail')
            print(f'Счёт {self._account_number}: ошибка снятия. {e}')

    def apply_interest(self, rate: float):
        """
        Начисление процентов на остаток (п.2 задания 2).
        Проверяет ставку >0, обновляет баланс, добавляет 'interest' в историю.
        """
        try:
            self._validate_amount(rate)
            interest_amount = self._balance * (rate / 100)
            self._balance += interest_amount
            self._add_operation('interest', interest_amount, 'success')
            print(f'Счёт {self._account_number}: Начислены проценты ({rate}%) в размере {interest_amount:.2f} у.е. Новый баланс: {self._balance:.2f} у.е.')
        except ValueError as e:
            print(f'Счёт {self._account_number}: Ошибка начисления процентов. {e}')

    def _get_valid_operation_types(self):
        """
        Переопределение для SavingsAccount (задание 3).
        Добавляет 'interest' к валидным типам операций.
        """
        return ['deposit', 'withdraw', 'interest']

In [615]:
pd.read_csv(r'C:\GitRepo\My-training-Slubik-Stanislav\transactions_dirty.csv')

,account_number,account_type,date,operation,amount,balance_after,status
0,ACC-100001,checking,2025-09-27 22:17:26,deposit,921.0,2121.00,success
1,ACC-100001,checking,2025-09-27 22:17:26,deposit,607.0,2728.00,success
2,ACC-100001,checking,2025-09-28 22:17:26,deposit,488.0,3216.00,success
3,ACC-100001,checking,28/09/2025 22:17,deposit,129.0,3345.00,success
4,ACC-100001,checking,2025-09-29 22:17:26,deposit,880.0,4225.00,success
...,...,...,...,...,...,...,...
100,ACC-100002,savings,09/10/2025 22:17,deposit,638.0,5241.27,success
101,ACC-100003,checking,2025-10-12 22:17:26,deposit,-115.0,3547.00,success
102,ACC-100004,savings,01/10/2025 22:17,withdraw,309.0,4247.00,success
103,ACC-100002,savings,2025-10-19 22:17:26,withdraw,300.0,6042.66,succes
